In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random

This version uses *tp, op* where *op* is granted up to 10 years after *tp*.

In [3]:
# pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
# .to_pandas()
# dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# # Get relevant US Patents
# pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]
# # Add MSA subfield
# pdf["msa_pc"] = list(zip(pdf["inv_msa"], pdf["primclass"]))

In [16]:
# Get number of primary class patents by MSA, and year of their first appearance
p2 = pdf.groupby(["inv_msa", "primclass"]).size()
# Only get primary classes with more than 50 patents
p2 = p2.loc[(p2 >= 50)].reset_index().rename(columns={0:"size"})
# Date of first appearance
p3 = pdf[["inv_msa", "primclass", "gyear"]].sort_values(["inv_msa", "primclass", "gyear"]).drop_duplicates(["inv_msa", "primclass"])
# Merge
p2 = p2.merge(p3, how="left", on=["inv_msa", "primclass"]).rename(columns={"gyear": "first_gyear"})

# New technology fields after 1990
p3 = p2.loc[(p2["first_gyear"] >= 1990)]

# Subfield
p3["msa_pc"] = list(zip(p3["inv_msa"], p3["primclass"]))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
# # tp: patent from new subfield
# # Primary class random sample
# ns = fastparquet.ParquetFile("DataStore/2018-07-P2/Reg0726/reg_naics_name_sim_tr_0726.parq").to_pandas()
# ns["op_msa_pc"] = list(zip(ns["op_inv_msa"], ns["op_primclass"]))
# ns["tp_msa_pc"] = list(zip(ns["tp_inv_msa"], ns["tp_primclass"]))
# ns = ns.loc[ns["tp_msa_pc"].isin(p3["msa_pc"])]

# Find highly similar PC
ns2 = ns.loc[(ns["norm_sim_mean_docvecs_pc_msa"] >= 1) &
             (ns["inv_msa_match"] == False), ["tp_msa_pc", "op_msa_pc",
"tp_primclass", "op_primclass", "tp_inv_msa", "op_inv_msa", "inv_msa_match", "year_group", 
"sim_mean_docvecs_pc_msa", "norm_sim_mean_docvecs_pc_msa"]].drop_duplicates()

# Comparison groups
ns3 = ns2[["tp_msa_pc", "op_msa_pc"]]
ns3["tp_first_gyear"] = ns3["tp_msa_pc"].map(dict(zip(p3["msa_pc"], p3["first_gyear"])))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [48]:
%%time
# Use only patents from relevant MSA-PC
p2 = pdf.loc[pdf["msa_pc"].isin(ns3["tp_msa_pc"]) | pdf["msa_pc"].isin(ns3["op_msa_pc"])]

# Patents by year
%time pats_by_yr = {n:g["patent"].tolist() for n,g in p2[["msa_pc", "gyear", "patent"]].groupby(["msa_pc", "gyear"])}

# Patents 10 years after
pats_by_10_yr = {}
for yr in range(1976,2010):
    p = p2.loc[p2["gyear"].isin(range(yr,yr+10))]
    d = {(n,yr): g["patent"].tolist() for n,g in p[["msa_pc", "patent"]].groupby(["msa_pc"])}
    pats_by_10_yr.update(d)

CPU times: user 5.62 s, sys: 0 ns, total: 5.62 s
Wall time: 5.66 s
CPU times: user 15.6 s, sys: 83.2 ms, total: 15.7 s
Wall time: 15.8 s


In [74]:
# Add a grant year for each of the 10 years following the first grant year of new subfield
ns4 = [list(zip([r1]*10, [r2]*10, [r3]*10, list(range(r3, r3+10)))) for r1,r2,r3\
       in zip(ns3["tp_msa_pc"], ns3["op_msa_pc"], ns3["tp_first_gyear"])]
ns4 = [item for sublist in ns4 for item in sublist]
ns4 = pd.DataFrame({"tp_msa_pc": [i[0] for i in ns4],
                   "op_msa_pc": [i[1] for i in ns4],
                   "tp_first_gyear": [i[2] for i in ns4],
                   "gyear": [i[3] for i in ns4],})

In [75]:
# Patents for each MSA-PC
ns4["tp_msa_pc_pats"] = [pats_by_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["tp_msa_pc"], ns4["gyear"])]
ns4["tp_msa_pc_pats_10"] = [pats_by_10_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["tp_msa_pc"], ns4["gyear"])]
ns4["op_msa_pc_pats"] = [pats_by_10_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["op_msa_pc"], ns4["gyear"])]
print(len(ns4))
# Drop whereever any of the patent lists are missing
ns4 = ns4.dropna(subset=['tp_msa_pc_pats', 'op_msa_pc_pats',],how="any")
print(len(ns4))

26250
19692


In [76]:
ns4.to_pickle("DataStore/2018-08/related_field_pats_0918.pkl")
ns4.head()

,gyear,op_msa_pc,tp_first_gyear,tp_msa_pc,tp_msa_pc_pats,op_msa_pc_pats
0,1991,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",[4999831],"[5007052, 5010544, 5012467, 5020058, 5043981, ..."
5,1996,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",[5535200],"[5487066, 5488608, 5491685, 5493562, 5495485, ..."
6,1997,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)","[5606553, 5636210]","[5592475, 5592486, 5594730, 5594734, 5596574, ..."
7,1998,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",[5742605],"[5706274, 5710756, 5717688, 5719862, 5721725, ..."
8,1999,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",[5940393],"[5856975, 5859837, 5859852, 5864545, 5867501, ..."


In [58]:
# Get sample pairs for each
rs = {}
%time msa_pc_pats = [list(itertools.product(i,j)) for i,j in zip(ns4["tp_msa_pc_pats"], ns4["op_msa_pc_pats"])]
# Add identifiers
%time msa_pc_pats = [list(zip(i, [(j,k,l,m)]*len(i))) for i,j,k,l,m in \
                    zip(msa_pc_pats, ns4["tp_msa_pc"], ns4["op_msa_pc"], ns4["gyear"], ns4["tp_first_gyear"])]
# Sample half
msa_pc_pats = (random.sample(l, int(np.round(len(l)/20))) for l in msa_pc_pats)
msa_pc_pats = [item[0]+item[1] for sublist in msa_pc_pats for item in sublist]


d = pd.DataFrame({"tp": [i[0] for i in msa_pc_pats],
                           "op": [i[1] for i in msa_pc_pats],
                            "tp_msa_pc": [i[2] for i in msa_pc_pats],
                            "op_msa_pc": [i[3] for i in msa_pc_pats],
                            "gyear": [i[4] for i in msa_pc_pats],
                            "tp_first_gyear": [i[5] for i in msa_pc_pats],})
d = d.drop_duplicates()
d = d.loc[~(d["tp"] == d["op"])]

# Drop same patent
rs["msa_pc_pats"] = d
print(len(rs["msa_pc_pats"]))

CPU times: user 13.5 s, sys: 3.78 s, total: 17.3 s
Wall time: 17.4 s
CPU times: user 25.5 s, sys: 2.09 s, total: 27.6 s
Wall time: 28.4 s
3109711


In [61]:
# msa_pc_pats = [list(itertools.product(i,j)) for i,j in zip(ns4["tp_msa_pc_pats"], ns4["op_msa_pc_pats"])]
# # Add identifiers
# msa_pc_pats = [list(zip(i, [(j,k,l,m)]*len(i))) for i,j,k,l,m in 
#                     zip(msa_pc_pats, ns4["tp_msa_pc"], ns4["tp_msa_pc"], ns4["gyear"], ns4["tp_first_gyear"])]
# # Sample half
# msa_pc_pats = (random.sample(l, int(np.round(len(l)/20))) for l in msa_pc_pats)
# msa_pc_pats = [item[0]+item[1] for sublist in msa_pc_pats for item in sublist]


d = pd.DataFrame({"tp": [i[0] for i in msa_pc_pats],
                           "op": [i[1] for i in msa_pc_pats],
                            "tp_msa_pc": [i[2] for i in msa_pc_pats],
                            "gyear": [i[4] for i in msa_pc_pats],
                            "tp_first_gyear": [i[5] for i in msa_pc_pats],})
d = d.drop_duplicates()
d = d.loc[~(d["tp"] == d["op"])]

rs["tp_msa_pc_pats"] = d

print(len(rs["tp_msa_pc_pats"]))

3109769


In [64]:
# Get similarity
def grouper(n, iterable):
    """
    >>> list(grouper(3, 'ABCDEFG'))
    [['A', 'B', 'C'], ['D', 'E', 'F'], ['G']]
    """
    iterable = iter(iterable)
    return iter(lambda: list(itertools.islice(iterable, n)), [])


import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]

print("Getting row values")
print(datetime.datetime.now())
pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
pat_dict = dict(zip(pat_dict, range(len(pat_dict))))

for dm in dms:
    print((dm,"started"))
    print("Loading matrix and dict")
    print(datetime.datetime.now())
    
    for k,v in rs.items():
    
        l2 = v.drop_duplicates()
        print(len(l2))
        # Store copy as array
        l3 = l2.loc[l2["tp"].isin(pat_dict.keys()) & l2["op"].isin(pat_dict.keys()), ["tp", "op"]].copy().drop_duplicates()

        if dm == "ldavecs":
            ncols = 60
        else:
            ncols = 100

        pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
    .to_pandas().values

        # Convert to chunks
        print("Getting chunks")
        print(datetime.datetime.now())
        # Split into chunks
        n_rows = 3000
        n_chunks = int(np.round(len(l3)/n_rows))
        tp_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["tp"].iteritems()]])
        op_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["op"].iteritems()]])
        del(pm)
        chunks = itertools.zip_longest(tp_chunks, op_chunks)

        print("Getting patent pair cosine similarity")
        print(datetime.datetime.now())
        # Cosine

        cos_dis = np.empty(len(l3))

        for r, c in enumerate(chunks):
            cos_dis[r*n_rows:r*n_rows+n_rows] = np.diag(distance.cdist(c[0],c[1], metric = "cosine"))

        l3["{0}_{1}".format(k, dm)] = 1-cos_dis

        # Rename columns
        l2 = l2.merge(l3.drop_duplicates(), how = "left", on = ["tp", "op"])
        del(l3)
        
        # Update results
        rs[k] = l2
        print(len(l2))
        print("finished")
        print(datetime.datetime.now())
    

Getting row values
2018-09-18 13:17:10.651913
('ldavecs', 'started')
Loading matrix and dict
2018-09-18 13:17:12.628021
3109711
Getting chunks
2018-09-18 13:17:38.208173
Getting patent pair cosine similarity
2018-09-18 13:18:03.681198
3109711
finished
2018-09-18 13:23:29.230606
3109769
Getting chunks
2018-09-18 13:23:42.857297
Getting patent pair cosine similarity
2018-09-18 13:24:04.289345
3109769
finished
2018-09-18 13:30:54.543800
('docvecs', 'started')
Loading matrix and dict
2018-09-18 13:30:54.544266
3109711
Getting chunks
2018-09-18 13:31:29.542682
Getting patent pair cosine similarity
2018-09-18 13:32:13.370671
3109711
finished
2018-09-18 13:40:51.207710
3109769
Getting chunks
2018-09-18 13:41:28.535275
Getting patent pair cosine similarity
2018-09-18 13:42:23.108830
3109769
finished
2018-09-18 13:50:54.788886


In [65]:
pickle.dump(rs, open("DataStore/2018-08/related_pair_sim_0918.pkl", "wb"))

In [69]:
r1 = rs["msa_pc_pats"][["tp_msa_pc", "op_msa_pc", "gyear", "msa_pc_pats_docvecs"]]\
.groupby(["tp_msa_pc", "op_msa_pc", "gyear"]).mean().reset_index()
r2 = rs["tp_msa_pc_pats"][["tp_msa_pc", "gyear", "tp_msa_pc_pats_docvecs"]]\
.groupby(["tp_msa_pc", "gyear"]).mean().reset_index()

In [70]:
ns4 = ns4.merge(r1, how="left", on = ["tp_msa_pc", "op_msa_pc", "gyear"])
ns4 = ns4.merge(r2, how="left", on = ["tp_msa_pc", "gyear"])

In [72]:
ns4.to_pickle("DataStore/2018-08/related_field_pats_0918.pkl")

In [73]:
ns4.columns

Index(['gyear', 'op_msa_pc', 'tp_first_gyear', 'tp_msa_pc', 'tp_msa_pc_pats',
       'op_msa_pc_pats', 'tp_msa_pc_pats_10', 'msa_pc_pats_docvecs',
       'tp_msa_pc_pats_docvecs'],
      dtype='object')